# Automated ML

# Dependencies

All the dependencies needed to complete the project appear here


In [1]:

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

from azureml.widgets import RunDetails

import joblib

from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


# Workspace
The config.json file is downloaded from Azure environment and has to be in the project folder in order for this cell to run

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-140390
aml-quickstarts-140390
southcentralus
aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee


# Create an Azure ML experiment

I am creating an experiment named machine-failure-experiment and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the source_directory for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the source_directory of the step.

In [3]:

# Choose a name for the run history container in the workspace.

experiment_name = 'heart-failure-automl'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

run = experiment.start_logging()

# Create or Attach a cluster

We will need to create a compute target for the AutoML run. In case the compute target (named compute-cluster in this script) is not found, a new one is created using the default AmlCompute as the training compute resource

#Configuring the Compute Cluster

In [4]:

# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-14T05:28:30.194000+00:00', 'errors': None, 'creationTime': '2021-03-14T04:59:13.037970+00:00', 'modifiedTime': '2021-03-14T04:59:28.807138+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


## Dataset

### Overview
The dataset is taken from kaggle : "https://www.kaggle.com/c/machine-failure-prediction/data?select=train.csv".



In [5]:

data = pd.read_csv('./heart_failure_clinical_records_dataset.csv')

found = False
key = "heart-failure-dataset"
description_text = "Prediction of patients’ survival with heart failure"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://raw.githubusercontent.com/gaya3arul/nd00333-capstone/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
                                
# Preview of the first five rows
print(data.head())

# Explore data
print(data.describe())

df = dataset.to_pandas_dataframe()
print(df.describe())

# Data columns
df.columns = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT']
x = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
y = df[['DEATH_EVENT']]

    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       

In [6]:
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration


AutoML Configuration
Here is an overview of the automl settings and configuration I used for the AutoML run:

"n_cross_validations": 2

This parameter sets how many cross validations to perform, based on the same number of folds (number of subsets). As one cross-validation could result in overfit, in my code I chose 2 folds for cross-validation; thus the metrics are calculated with the average of the 2 validation metrics.

"primary_metric": 'accuracy'

I chose accuracy as the primary metric as it is the default metric used for classification tasks.

"enable_early_stopping": True

It defines to enable early termination if the score is not improving in the short term. In this experiment, it could also be omitted because the _experiment_timeoutminutes is already defined below.

"max_concurrent_iterations": 4

It represents the maximum number of iterations that would be executed in parallel.

"experiment_timeout_minutes": 20

This is an exit criterion and is used to define how long, in minutes, the experiment should continue to run. To help avoid experiment time out failures, I used the value of 20 minutes.

"verbosity": logging.INFO

The verbosity level for writing to the log file.

compute_target = compute_target

The Azure Machine Learning compute target to run the Automated Machine Learning experiment on.

task = 'classification'

This defines the experiment type which in this case is classification. Other options are regression and forecasting.

training_data = dataset

The training data to be used within the experiment. It should contain both training features and a label column - see next parameter.

label_column_name = 'DEATH_EVENT'

The name of the label column i.e. the target column based on which the prediction is done.

path = project_folder

The full path to the Azure Machine Learning project folder.

featurization = 'auto'

This parameter defines whether featurization step should be done automatically as in this case (auto) or not (off).

debug_log = 'automl_errors.log

The log file to write debug information to.

enable_onnx_compatible_models = False

I chose not to enable enforcing the ONNX-compatible models at this stage. However, I will try it in the future. For more info on Open Neural Network Exchange (ONNX), please see here.

In [7]:

# Automl settings

automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

# Parameters for AutoMLConfig

automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=True,
                            blocked_models=['XGBoostClassifier'],
                            **automl_settings
                            )

In [8]:

# Submit the experiment

remote_run = experiment.submit(automl_config, show_output = True)
remote_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

{'runId': 'AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-14T05:40:28.571254Z',
 'endTimeUtc': '2021-03-14T05:59:14.701212Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-automl","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-140390","workspace_name":"quick-starts-ws-140390","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":null,"y_max":null,"num_classes":

## Run Details


In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
# Fetch the latest status of the run. It should show 'Completed'

print("Run Status: ",remote_run.get_status())

Run Status:  Completed


# Run Details
In the cell below, I use the RunDetails widget and show the children runs of the experiment.

In [10]:
RunDetails(remote_run).show()

# Get details from each run
for child_run in remote_run.get_children():
    print('===================================================')
    print(child_run)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_42,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_40,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_41,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_39,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_38,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_37,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_36,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-49

## Best Model




In [11]:


automl_best_run, fitted_automl_model = remote_run.get_output() 
print("Fitted model: ", fitted_automl_model.steps[-1])
print("---"*5)
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("---"*5)
print("AutoML run Summary: ", remote_run.summary())

Package:azureml-automl-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-core, training version:1.23.0, current version:1.22.0
Package:azureml-dataprep, training version:2.10.1, current version:2.9.1
Package:azureml-dataprep-native, training version:30.0.0, current version:29.0.0
Package:azureml-dataprep-rslex, training version:1.8.0, current version:1.7.0
Package:azureml-dataset-runtime, training version:1.23.0, current version:1.22.0
Package:azureml-defaults, training version:1.23.0, current version:1.22.0
Package:azureml-interpret, training version:1.23.0, current version:1.22.0
Package:azureml-mlflow, training version:1.23.0, current version:1.22.0
Package:azureml-pipeline-core, training version:1.23.0, current version:1.22.0
Package:azureml-telemetry, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-client, training version:1.23.0, current version:1.22.0
Package:azureml-train-automl-runtime, training version:1.23.0, current versio

Fitted model:  ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('25',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f805c7ee7b8>),
                                                           ('randomforestclassifier',
                                                            RandomForestClassifier(bootstrap=False,
                                                                                   ccp_alpha=0.0,
                                                                                   class_weight=None,
                                                                                   criterion='entropy',
                                                

## Transparency

View updated featurization summary

In [12]:

custom_featurizer = fitted_automl_model.named_steps['datatransformer']
df = custom_featurizer.get_featurization_summary()
pd.DataFrame(data=df)

,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations
0,age,Numeric,No,1,[MeanImputer]
1,creatinine_phosphokinase,Numeric,No,1,[MeanImputer]
2,ejection_fraction,Numeric,No,1,[MeanImputer]
3,platelets,Numeric,No,1,[MeanImputer]
4,serum_creatinine,Numeric,No,1,[MeanImputer]
5,serum_sodium,Numeric,No,1,[MeanImputer]
6,time,Numeric,No,1,[MeanImputer]
7,anaemia,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]
8,diabetes,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]
9,high_blood_pressure,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]


In [13]:

df = custom_featurizer.get_featurization_summary(is_user_friendly=False)
pd.DataFrame(data=df)

,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations,TransformationParams
0,age,Numeric,No,1,[MeanImputer],"{'Transformer1': {'Input': ['age'], 'Transform..."
1,creatinine_phosphokinase,Numeric,No,1,[MeanImputer],{'Transformer1': {'Input': ['creatinine_phosph...
2,ejection_fraction,Numeric,No,1,[MeanImputer],{'Transformer1': {'Input': ['ejection_fraction...
3,platelets,Numeric,No,1,[MeanImputer],"{'Transformer1': {'Input': ['platelets'], 'Tra..."
4,serum_creatinine,Numeric,No,1,[MeanImputer],{'Transformer1': {'Input': ['serum_creatinine'...
5,serum_sodium,Numeric,No,1,[MeanImputer],"{'Transformer1': {'Input': ['serum_sodium'], '..."
6,time,Numeric,No,1,[MeanImputer],"{'Transformer1': {'Input': ['time'], 'Transfor..."
7,anaemia,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder],"{'Transformer1': {'Input': ['anaemia'], 'Trans..."
8,diabetes,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder],"{'Transformer1': {'Input': ['diabetes'], 'Tran..."
9,high_blood_pressure,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder],{'Transformer1': {'Input': ['high_blood_pressu...


 ## Retrieve the Best Model's explanation
Retrieve the explanation from the best_run which includes explanations for engineered features and raw features. Make sure that the run for generating explanations for the best model is completed.

In [17]:

# Wait for the best model explanation run to complete
from azureml.core.run import Run
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()



AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_ModelExplain


{'runId': 'AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_ModelExplain',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-14T05:59:26.783224Z',
 'endTimeUtc': '2021-03-14T06:03:22.183875Z',
 'properties': {'azureml.runsource': 'automl',
  'parentRunId': 'AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_42',
  '_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '5c7caf56-bbdb-4db3-b598-9557bf74eb78',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'dependencies_versions': '{"azureml-train-automl-runtime": "1.23.0", "azureml-train-automl-client": "1.23.0", "azureml-telemetry": "1.23.0", "azureml-pipeline-core": "1.23.0", "azureml-model-management-sdk": "1.0.1b6.post1", "azureml-mlflow": "1.23.0", "azureml-interpret": "1.23.0", "azureml-defaults": "1.23.0", "azureml-dataset-runtime": "1.23.0", "azureml-dataprep": "2.10.1", "azureml-dataprep-rslex": "1.8.0", "azureml-dataprep-native": "30.

# Download engineered feature importance from artifact store
You can use ExplanationClient to download the engineered feature explanations from the artifact store of the best_run.

In [19]:
!pip install azureml-interpret

In [21]:
from azureml.interpret import ExplanationClient
client = ExplanationClient.from_run(automl_best_run)
engineered_explanations = client.download_model_explanation(raw=False)
exp_data = engineered_explanations.get_feature_importance_dict()
exp_data

{'time_MeanImputer': 0.7972626067294868,
 'ejection_fraction_MeanImputer': 0.3034245382595435,
 'serum_creatinine_MeanImputer': 0.27519197597162437,
 'age_MeanImputer': 0.14115282582667082,
 'serum_sodium_MeanImputer': 0.09896010423219213,
 'platelets_MeanImputer': 0.07939855898092651,
 'creatinine_phosphokinase_MeanImputer': 0.05320176959329277,
 'anaemia_ModeCatImputer_LabelEncoder': 0.012763838217101637,
 'sex_ModeCatImputer_LabelEncoder': 0.008619860783620505,
 'smoking_ModeCatImputer_LabelEncoder': 0.008612374243420266,
 'diabetes_ModeCatImputer_LabelEncoder': 0.004080903604352678,
 'high_blood_pressure_ModeCatImputer_LabelEncoder': 0.002684655259114307}


## Retrieve the Best ONNX Model
Below we select the best pipeline from our iterations. The get_output method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing. Overloads on get_output allow you to retrieve the best run and fitted model for any logged metric or for a particular iteration.

Set the parameter return_onnx_model=True to retrieve the best ONNX model, instead of the Python model.

In [22]:
best_run, onnx_mdl = remote_run.get_output(return_onnx_model=True)

## Save the best ONNX model

In [23]:

from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

In [24]:
automl_best_run.get_file_names()

# Download the yaml file that includes the environment dependencies
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [25]:
# Download the model file

automl_best_run.download_file('outputs/model.pkl', 'Automl_model.pkl')

In [26]:
print(fitted_automl_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.056842105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=400,
    

In [27]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-automl,AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_42,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
# Save the best model
automl_best_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')

print(automl_best_run)

Run(Experiment: heart-failure-automl,
Id: AutoML_f1aafa23-4cfa-496b-8db6-30ac5a0eca6c_42,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment



As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

In [29]:

model = remote_run.register_model(model_name = 'best_run_automl.pkl')
print(remote_run.model_id)


environment = automl_best_run.get_environment()
entry_script='inference/scoring.py'
automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

# Deploying the model via ACI WebService
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-deploy-azure-container-instance.md

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "deploy-heart-failure-automl", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

best_run_automl.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [30]:

# Getting the service state
# The scorig URI & the primary authentication key are copied to the endpoint.py file in order to test the deployed service.
# The Swagger URI can be used in Swagger UI: https://petstore.swagger.io/ For more info, please see the relevant part in the README file.

# Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)

Service state: Healthy
Service scoring URI: http://68166165-2e3a-4314-b3af-0097d0758ba5.southcentralus.azurecontainer.io/score
Service Swagger URI: http://68166165-2e3a-4314-b3af-0097d0758ba5.southcentralus.azurecontainer.io/swagger.json
Service primary authentication key: 3BgoDzQDTSlWh6hZe7LpoQX9vpAXF1Vt


In [31]:
# Sending a request to the deployed web service to test it: consuming model endpoint

%run endpoint.py

{"result": [1, 1]}


In [32]:

# Printing the logs
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-03-14T06:25:27,847214600+00:00 - gunicorn/run 
2021-03-14T06:25:27,848049400+00:00 - rsyslog/run 
2021-03-14T06:25:27,859860900+00:00 - iot-server/run 
2021-03-14T06:25:27,886911900+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

# Deleting the service
Putting the deletion of the service in a separate cell to avoid accidentally running the cell before finishing the tasks

In [ ]:
service.delete()

In [ ]:
conda install-c anaconda py-xgboost==0.90